In [1]:
import sys
import os

import utils

import importlib

from preprocess import main as preprocess_nd2
from mip import main as mip

_ = importlib.reload(sys.modules['mip'])
_ = importlib.reload(sys.modules['utils'])

# SPECIFY DATA TO PROCESS

In [2]:

DATA_PTH = r'C:\Users\munib\POSTDOC\DATA\g5ht-free\20251028'
INPUT_ND2 = 'date-20251028_time-1500_strain-ISg5HT_condition-starvedpatch_worm001.nd2'
INPUT_ND2_PTH = os.path.join(DATA_PTH, INPUT_ND2)
NOISE_PTH = r'C:\Users\munib\POSTDOC\CODE\g5ht-pipeline\noise_042925.tif'

OUT_DIR = os.path.splitext(INPUT_ND2_PTH)[0]

STACK_LENGTH = 41

noise_stack = utils.get_noise_stack(NOISE_PTH, STACK_LENGTH)
stack_range = utils.get_range_from_nd2(INPUT_ND2_PTH) # aka frame indices

# 1. PREPROCESS

In [4]:
from preprocess import main as preprocess_nd2
_ = importlib.reload(sys.modules['utils'])

# command-line arguments
sys.argv = ["", INPUT_ND2_PTH, "0", NOISE_PTH]

# Call the main function
preprocess_nd2()

Stack 0000 shear corrected
Stack 0000 aligned


### TEST PARALLEL

In [5]:
from preprocess_parallel import main as preprocess_nd2_parallel
_ = importlib.reload(sys.modules['preprocess_parallel'])

# command-line arguments
sys.argv = ["", INPUT_ND2_PTH, "0", "1199", NOISE_PTH]

# Call the main function
preprocess_nd2_parallel()

Processing 1200 stacks (0-1199) using 24 workers...


100%|██████████| 1200/1200 [1:27:17<00:00,  4.36s/it] 


Parallel preprocessing complete.


# 2. MAX INTENSITY PROJECTION

In [7]:
from mip import main as mip

_ = importlib.reload(sys.modules['mip'])
_ = importlib.reload(sys.modules['utils'])

# command-line arguments
sys.argv = ["", INPUT_ND2_PTH]

# Call the main function
mip()

<tifffile.TiffWriter 'mip.tif'> writing nonconformant BigTIFF ImageJ


0000.tif written to mip.tif
0001.tif written to mip.tif
0002.tif written to mip.tif
0003.tif written to mip.tif
0004.tif written to mip.tif
0005.tif written to mip.tif
0006.tif written to mip.tif
0007.tif written to mip.tif
0008.tif written to mip.tif
0009.tif written to mip.tif
0010.tif written to mip.tif
0011.tif written to mip.tif
0012.tif written to mip.tif
0013.tif written to mip.tif
0014.tif written to mip.tif
0015.tif written to mip.tif
0016.tif written to mip.tif
0017.tif written to mip.tif
0018.tif written to mip.tif
0019.tif written to mip.tif
0020.tif written to mip.tif
0021.tif written to mip.tif
0022.tif written to mip.tif
0023.tif written to mip.tif
0024.tif written to mip.tif
0025.tif written to mip.tif
0026.tif written to mip.tif
0027.tif written to mip.tif
0028.tif written to mip.tif
0029.tif written to mip.tif
0030.tif written to mip.tif
0031.tif written to mip.tif
0032.tif written to mip.tif
0033.tif written to mip.tif
0034.tif written to mip.tif
0035.tif written to 

# 3. SEGMENT

`conda activate segment-torch`

Uses a separate conda environment from the rest of the pipeline. create it using:
`conda env create -f segment_torch.yml`

In [1]:
import sys
import os

In [2]:
DATA_PTH = r'C:\Users\munib\POSTDOC\DATA\g5-HT-free\022025_eft_41z_starved_worm002'
INPUT_ND2 = '022025_eft_41z_starved_worm002.nd2'
INPUT_ND2_PTH = os.path.join(DATA_PTH, INPUT_ND2)
NOISE_PTH = r'C:\Users\munib\POSTDOC\CODE\g5ht-pipeline/noise_042925.tif'

OUT_DIR = os.path.splitext(INPUT_ND2_PTH)[0]

In [3]:
OUT_DIR

'C:\\Users\\munib\\POSTDOC\\DATA\\g5-HT-free\\022025_eft_41z_starved_worm002\\022025_eft_41z_starved_worm002'

In [4]:
from segment.segment import main as segment_mip

MIP_PTH = os.path.join(OUT_DIR, 'mip.tif')

# command-line arguments
sys.argv = ["", MIP_PTH]

segment_mip()


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\munib\miniconda3\envs\segment_cpu\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\munib\miniconda3\envs\segment_cpu\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "c:\Users\munib\miniconda3\envs\segment_cpu\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\munib\miniconda3\envs\segment_cpu\lib\site-packages\traitlets\config\application.py", line 1075, in launc

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\munib\miniconda3\envs\segment_cpu\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\munib\miniconda3\envs\segment_cpu\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "c:\Users\munib\miniconda3\envs\segment_cpu\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\munib\miniconda3\envs\segment_cpu\lib\site-packages\traitlets\config\application.py", line 1075, in launc

AttributeError: _ARRAY_API not found

ImportError: numpy.core._multiarray_umath failed to import

ImportError: numpy.core.umath failed to import

TypeError: Unable to convert function return value to a Python type! The signature was
	() -> handle